## Introduction (15-688 only)
In this problem, you're going to write a (very minimal) graph library, which uses both the adjacency dictionary and the sparse adjacency matrix representation of a graph.  Using these two representations, you'll implement two of the more well-known large-scale graph processing algorithms: Djisktra's algorithm for finding single-source shortest paths in the graph, and the PageRank algorithm for determining the importance of nodes in the network.

## The test graph

The graph we'll be focusing on for most of this assignment (and which we'll also be grading you on, we'll just be using a subset of the nodes that you won't know in advance), is a directed graph that represents the page links in the English language Wikipedia.  Specifically, we took the (pre-processed) Wikipedia dump from here: http://haselgrove.id.au/wikipedia.htm , which were taken from a 2008 version of Wikipedia, and we then selected only subselected only those nodes that had at least _500 incoming links_.  This resulted in a graph with about 24 thousands that had about 6 million edges between the nodes.

## Q0: Warmup using networkx

In this first part of the question: which will _not_ be graded (in the submitted version, you can't include the `networkx` library), you'll load the graph using `networkx` and compute shortest distances from a source node.  There are two files included with this notebook that are relevant here:

    wikipedia_small.graph
    wikipedia_small.nodes
    
The `.graph` file contains a list of integers, two per each line.  If the line "`i j`" appears in the file, this indicates a directed edge from node `i` to node `j`.  The `.nodes` file then contains a list of each node the the graph, where the link number indicates the node index.  This is how we can relate the node numbers in the `.graph` file to actual pages on Wikipedia.

For Q0, load this graph into a networkx `DiGraph` object.  Note that the nodes in a networkx graph can be any hashable type (ints, strings, etc), so you can use any of these as a potential for the nodes (you could load loads corresponding to the integer indices from the `.graph` file, string versions of indices, or the node names themselves.  However you do it, though, you'll want to make sure that you can determine the actual Wikipedia page from the node number, and vice versa. (For what it's worth, we used the integer indices as node keys, and then created a separate dictionary that could look up the index from a node name).

After you have loaded the graph, use the `nx.shortest_path_length()` function to get the link distance from the `Carnegie_Mellon_University` node to all other nodes in the network.

In [277]:
# AUTOLAB_IGNORE_START
import networkx as nx

In [278]:
# AUTOLAB_IGNORE_START
with open("wikipedia_small.graph", "r") as f:
    wiki_edgelst = [line.split(" ") for line in f.read().split("\n")]
with open("wikipedia_small.nodes", "r") as f:
    nodelst = [line for line in f.read().split("\n")]
nodedict = dict(enumerate(nodelst))
# AUTOLAB_IGNORE_STOP

In [279]:
# AUTOLAB_IGNORE_STOP

When we run this call, we get a maximum hop count of 5 to any other node in the network (or infinity, some nodes cannot be reached by our node, usually ones that have only outlinks but no inlinks from other nodes in the network, but networkx does not return these in the `shortest_path_length()` result).  One such node is the `List_of_Salticidae_species` page, which gets connected via the path (which you can find via `nx.shortest_path()`:

    Carnegie_Mellon_University
    California
    Gasoline
    Blood
    Jumping_spider
    List_of_Salticidae_species

(note that not all these links appear to still occur in the current version of Wikipedia).  There are also calls in newtorkx that retrieve and adjacency matrix and run the PageRank algorithm, but these are quite slow, and would take too long to run on a graph this size.

## Your own Graph class

In the main portion of this assignment, you'll create your own Graph class that mimics some of the functionality of networkx (and which will indeed be much faster than networkx when it comes to algorithms like PageRank).  Here is the template for your Graph call, which you'll fill in throughout the assignment.

In [280]:
import numpy as np
import scipy.sparse as sp
import heapdict

class Graph:
    def __init__(self):
        """ Initialize with an empty edge dictionary. """
        self.edges = {}
        self.nodes = set()
        self.A = False
        self.nlist = []
    
    def add_edges(self, edges_list):
        """ Add a list of edges to the network.
        
        Args:
            edges_list: list of (a,b) tuples, where a->b is an edge to add
        """       
        for left, right in edges_list:
            if left in self.edges:
                 self.edges[left].update({right:1.0})
            else:
                self.nodes.add(left)
                self.edges[left] = {right:1.0}
            
            if right not in self.edges:
                self.nodes.add(right)                
                self.edges[right] ={}
                

        
    def get_neighbors(self, node):
        neighbors = set(self.edges[node].keys())
        return neighbors
    
    def shortest_path(self, source):
        """ Compute the single-source shorting path.
        
        This function uses Djikstra's algorithm to compute the distance from 
        source to all other nodes in the network.
        
        Args:
            source: node index for the source
            
        Returns: tuple: dist, path
            dist: dictionary of node:distance values for each node in the graph, 
                  where distance denotes the shortest path distance from source
            path: dictionary of node:prev_node values, where prev_node indicates
                  the previous node on the path from source to node
        """
        
        Q = self.nodes.copy()
        dist = {}
        path = {}

        for vertex in Q:
            dist[vertex] = float("inf")
            path[vertex] = None

        # Distance from source to itself
        dist[source] = 0 

        hdist = heapdict.heapdict(dist)                       

        while Q:
            
            debugdict = self.__dict__

            # get subset of dist dict where key in Q
#             distQ = { key:value for key,value in dist.items() if key in Q }

            # find key with smallest dist 
            u = hdist.popitem()[0]
            
            #remove smallest dist key from set Q
            Q.remove(u)
            
            # find neighbors of u
            neighbors = self.get_neighbors(u)
            
            # filter for neighbors in Q
            neighbors = neighbors.intersection(Q)

            for neighbor in neighbors:
                alt = dist[u] + 1
                if alt < dist[neighbor]:
                    dist[neighbor] = alt 
                    hdist[neighbor] = alt 
                    path[neighbor] = u 

        return dist, path

        
    def adjacency_matrix(self):
        """ Compute an adjacency matrix form of the graph.  
        
        Returns: tuple (A, nodes)
            A: a sparse matrix in COO form that represents the adjaency matrix
               for the graph (i.e., A[j,i] = 1 iff there is an edge i->j)
               NOTE: be sure you have this ordering correct!
            nodes: a list of nodes indicating the node key corresponding to each
                   index of the A matrix
        """
        nodes = list(self.nodes)
        nodeIndexDict = {node:i for i,node in enumerate(nodes)}
        col  = np.array([nodeIndexDict[src_key] for src_key, valdict in self.edges.iteritems() for trgt_key in valdict.iterkeys()])
        row  = np.array([nodeIndexDict[trgt_key] for src_key, valdict in self.edges.iteritems() for trgt_key in valdict.iterkeys()])

        data = np.ones(len(row), dtype=np.int)
        
        A = sp.coo_matrix((data, (row,col)))
        
        self.A = A
        self.nlist = nodes
        return (A, nodes)
    
    def pagerank(self, d=0.85, iters=100):
        """ Compute the PageRank score for each node in the network.
        
        Compute PageRank scores using the power method.
        
        Args:
            d: 1 - random restart factor
            iters: maximum number of iterations of power method
            
        Returns: dict ranks
            ranks: a dictionary of node:importance score, for each node in the
                   network (larger score means higher rank)
        
        """
        A, nlist = self.adjacency_matrix()
        
        sumA = np.array(A.sum(axis=0))[0]
        
        invSumA = 1.0/sumA
        
        xpd_invSumA = np.array([invSumA[col] for col in A.col])
        A.data = A.data*xpd_invSumA*d
        
        # intialize ranks to uniform dist
        rank = np.array([1.0/len(nlist)]*len(nlist))

        for i in range(iters):
            rank = A.dot(rank) + (1-d)*1.0/len(nlist)
        
        rank = rank/sum(rank)
        ranks = dict(zip(nlist,rank))
        return ranks
        
         
# HANDOUT_END  

In [281]:
# AUTOLAB_IGNORE_START
G = Graph()
G.add_edges([("A","B"), ("B","C"), ("A","D"), ("D", "E"), ("E", "B")])
print G.edges

G.get_neighbors("A")

# AUTOLAB_IGNORE_STOP

{'A': {'B': 1.0, 'D': 1.0}, 'C': {}, 'B': {'C': 1.0}, 'E': {'B': 1.0}, 'D': {'E': 1.0}}


{'B', 'D'}

In [282]:
# AUTOLAB_IGNORE_START

{"A":{"B":1.0,
    "D":1.0}, 
 "B":{"C":1.0},
 "C":{},
 "D":{"E":1.0}, 
 "E":{"B":1.0}}

# AUTOLAB_IGNORE_STOP

{'A': {'B': 1.0, 'D': 1.0},
 'B': {'C': 1.0},
 'C': {},
 'D': {'E': 1.0},
 'E': {'B': 1.0}}

## Q1: Implement `add_edges()` call

To begin, implement the function `add_edges()`.  This will modify the `self.edges` variable to add all edges passed as tuples in `edges_list`.  Note that `self.edges` should be represented as an "adjacency dictionary", so that for every node `i` in the graph

    self.edges[i]
    
indicators another dictionary of nodes that `i` is connected to (with the direction pointing from `i` to the other node).  For instance, if `self.edges[i][j]` is a valid entry this means that there is an edge from node `i` to node `j` (the value of this entry doesn't matter, so we could technically use a dictionary of sets, but we use a dictionary of dictionaries to keep things a little bit more uniform and to allow for potential extensions e.g. to weighted graphs).

For instance, the following code populates a simple graph like the following:

    A -> B -> C
    |    ^
    v    |
    D -> E

In [283]:
# AUTOLAB_IGNORE_START
G = Graph()
G.add_edges([("A","B"), ("B","C"), ("A","D"), ("D", "E"), ("E", "B")])
print G.edges
# AUTOLAB_IGNORE_STOP

{'A': {'B': 1.0, 'D': 1.0}, 'C': {}, 'B': {'C': 1.0}, 'E': {'B': 1.0}, 'D': {'E': 1.0}}


In addition, make sure your code is able to load the Wikipedia graph from above.  For reference, our routine takes about 22 second to load the full graph.

## Q2: Djisktra's algorithm

Next, implement Djisktra's single-source shortest path algorithm (with the simple case where the distance along any edge is assumed to be one).  You can refer to the Wikipedia page on Djikstra's algorithm for reference (https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm), but the basic idea of the algorithm is to keep a priority queue of the current known distances to a node.  We continually pop off the smallest element `i` in the queue, and then update all its successor nodes to have a distance of 1 + distance[i].

For the priority queue, you should use the heapdict library (https://github.com/DanielStutzbach/heapdict) that is included above (and included with Anaconda).  You can update the priority of a heapdict element and pop off the smallest element using the syntax

    d = heapdict({"a":5, "b":6})
    d["b"] = 4
    d.popitem() # -> ("b", 4)
    ...
    
The function should return a list both of the shortest distances and the previous nodes in the shortest path from the source node to this node.  For instance, executeing the algorithm on our graph above gives the following result.
    

In [286]:
# AUTOLAB_IGNORE_START
print "A", G.shortest_path("A")
print "B", G.shortest_path("B")
print "C", G.shortest_path("C")
print "D", G.shortest_path("D")
print "E", G.shortest_path("E")
# AUTOLAB_IGNORE_STOP

A ({'A': 0, 'C': 2, 'B': 1, 'E': 2, 'D': 1}, {'A': None, 'C': 'B', 'B': 'A', 'E': 'D', 'D': 'A'})
B ({'A': inf, 'C': 1, 'B': 0, 'E': inf, 'D': inf}, {'A': None, 'C': 'B', 'B': None, 'E': None, 'D': None})
C ({'A': inf, 'C': 0, 'B': inf, 'E': inf, 'D': inf}, {'A': None, 'C': None, 'B': None, 'E': None, 'D': None})
D ({'A': inf, 'C': 3, 'B': 2, 'E': 1, 'D': 0}, {'A': None, 'C': 'B', 'B': 'E', 'E': 'D', 'D': None})
E ({'A': inf, 'C': 2, 'B': 1, 'E': 0, 'D': inf}, {'A': None, 'C': 'B', 'B': 'E', 'E': None, 'D': None})


This indicates, for instance, that the shortest path from A to C has length 2 and is given by the path `A -> B -> C`.  You should also test your function on the wikipedia graph and verify that it gives the same results as in networkx version.

## Q3: Adjacency matrix representation

Implement the adjacency matrix method of the Graph class.  This returns a matrix representing the adjacency of the graph (in scipy COO sparse format), as well as a list of nodes that indicate how the indices in this graph relate to the nodes in the network.

IMPORTANT: in order to complete this question in a manner that works on the Wikipedia, you will _need_ to implement this function natively as a sparse matrix (i.e., you cannot construct a dense matrix and then convert that to a sparse matrix, but need to directly use the `sp.coo_matrix()` constructor).  The Wikipedia graph is is 24K x 24K nodes, which (assuming 8 bytes per entry, would take up 4GB of memory.  While it's not impossible to do things this way at this scale (it quickly becomes infeasible for graphs that are even slightly larger), it's a very bad idea, and just allocating this much memory will take too long.

For example, in our graph above we have the following:

In [270]:
# AUTOLAB_IGNORE_START
A, nlist = G.adjacency_matrix()
print type(A)
print A.todense()
print nlist
# AUTOLAB_IGNORE_STOP

<class 'scipy.sparse.coo.coo_matrix'>
[[0 0 0 0 0]
 [0 0 1 0 0]
 [1 0 0 1 0]
 [0 0 0 0 1]
 [1 0 0 0 0]]
['A', 'C', 'B', 'E', 'D']


Again, make sure your code works on the Wikipedia graph.  In our implementation, it takes 9 seconds to run on this graph.

## Q4: PageRank algorithm

Finally, implement the PageRank algorithm using the adjacency matrix representation that you built in the previous question.  Reference on the PageRank algorithm is here: https://en.wikipedia.org/wiki/PageRank.  You should specifically use the approach described in the "Power Method" section on this page, which we also discussed in class.  This involves forming some initial uniform probability vector $x$, and repeatly multiplying it by the matrices:
\begin{equation}
x := (d P + ((1-d)/n) E)x
\end{equation}
where $P$ is a transition matrix (the $A$ matrix normalized so that all columns have sum 1), $E$ is the matrix of all ones, and $d$ is the damping factor.

NOTES: Recall that from the definition of PageRank, when we reach a "sink" node (a node with no outgoing edges), we randomly hope to any other node in the network, so that columns of $P$ that have no outgoing edges are set to the uniform distribution.  To be efficient, you'll also want to avoid explicitly forming the $E$ matrix, and should instead use the fact that $E = 11^T$ where $1$ denotes a vector of all ones.  Use the fact that we can reorder matrix multiplication if associative (i.e., the fact the $A(BC)$ = $(AB)C$) to make this operation as fast as possible.

Your function should return a dictionary of nodes and their corresponding page rank.  For example, in our graph above, we have the following reults:

In [264]:
# AUTOLAB_IGNORE_START
G.pagerank()


# AUTOLAB_IGNORE_STOP

C:\Anaconda3\envs\dato-env\lib\site-packages\ipykernel\__main__.py:147: RuntimeWarning: divide by zero encountered in divide


{'A': 0.085108623870681688,
 'B': 0.28124676686965955,
 'C': 0.32416837570989232,
 'D': 0.12127978901572142,
 'E': 0.18819644453404491}

In [265]:
# AUTOLAB_IGNORE_START
W = Graph()
W.add_edges(wiki_edgelst)
# AUTOLAB_IGNORE_STOP

In [266]:
# AUTOLAB_IGNORE_START
wikirank_raw = W.pagerank()
wikirank = {nodedict[int(k)]:v for k, v in wikirank_raw.items()}
# AUTOLAB_IGNORE_STOP

C:\Anaconda3\envs\dato-env\lib\site-packages\ipykernel\__main__.py:147: RuntimeWarning: divide by zero encountered in divide


In [268]:
# AUTOLAB_IGNORE_START
result = sorted(wikirank.items(), key=lambda x: x[1], reverse=True)
print sum(r[1] for r in result)
result[0:15]
# AUTOLAB_IGNORE_STOP

1.0


[('United_States', 0.0027518870536402955),
 ('2007', 0.0024433905407855007),
 ('2008', 0.0021734251483718826),
 ('Wikimedia_Commons', 0.0017260420017522586),
 ('United_Kingdom', 0.0015999885109100329),
 ('2006', 0.0015484491173122946),
 ('France', 0.0014495899414189285),
 ('Wiktionary', 0.0012675347642074604),
 ('Canada', 0.0010989619526347431),
 ('World_War_II', 0.0010491307968022643),
 ('2005', 0.001046330246169557),
 ('List_of_African_films', 0.0010071387023193896),
 ('Germany', 0.00095626219282239134),
 ('Europe', 0.00093769002565240655),
 ('English_language', 0.00090814436025853952)]

As is intuitive, nodes B and C have higher page rank, as they are pointed to by more of the other nodes.

Make sure your implementation works on the full Wikipedia graph (in our implementation, it takes 11 seconds to run, most of which is taken up by generating the adjacency matrix).  The top PageRank entires we get from our implementation are (drumroll...)

    United_States 0.00275188705264
    2007 0.00244339053989
    2008 0.00217342514773
    Wikimedia_Commons 0.00172604200122
    United_Kingdom 0.00159998851013
    2006 0.00154844911674
    France 0.00144958994072
    Wiktionary 0.00126753476354
    Canada 0.00109896195215
    World_War_II 0.00104913079624
    2005 0.00104633024568
    List_of_African_films 0.00100713870383
    Germany 0.000956262192248
    Europe 0.000937690025073
    English_language 0.000908144359626
    Geographic_coordinate_system 0.000891711151403
    Latin 0.000888662228804
    Australia 0.000879854710005
    India 0.000787625093175
    Japan 0.000784815389935

countries and years! (and wikipedia links and a couple other rather interesting items).  But overall a seemingly reasonable list of pages we may expect to be important.

